In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from step import preprocessing as pp
from step import osm
from step.model import step, namespaces
from step.model import utility

import json
import gpxpy
import pandas as pnd
import numpy as np

import shapely
from shapely import wkt
from shapely.ops import polygonize_full, cascaded_union
import folium
import geojson
import warnings

import rdflib
from rdflib import URIRef, Namespace, Graph
from SPARQLWrapper import SPARQLWrapper2

%load_ext autoreload
%autoreload 2

In [2]:
_1 = r'C:\DATA\MapMyRun\Grenoble\40890583\16_16_Run _ Jog\950590555.gpx'
_2 = r'C:\DATA\MapMyRun\Grenoble\37695677\16_16_Run _ Jog\1007756961.gpx'
_3 = r'C:\DATA\MapMyRun\Grenoble\49996348\16_16_Run _ Jog\1079694145.gpx'
_4 = r'C:\DATA\MapMyRun\Grenoble\27031614\16_283_Easy _ General\1223879101.gpx'
_5 = r'C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1320895325.gpx'
_6 = r'C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1234176355.gpx'
_7 = r'C:\DATA\MapMyRun\Grenoble\27031614\16_16_Run _ Jog\728010151.gpx'
_8 = r'C:\DATA\MapMyRun\Rio\11346850\16_16_Run _ Jog\987162013.gpx'
_9 = r'C:\DATA\MapMyRun\Rio\9612880\16_16_Run _ Jog\1080026447.gpx'
_10 = r'C:\DATA\MapMyRun\Rio\42350047\16_16_Run _ Jog\909856727.gpx'
_11 = r'C:\DATA\MapMyRun\Rio\17670613\16_16_Run _ Jog\1053150801.gpx'
_12 = r'C:\DATA\MapMyRun\Rio\17670613\16_16_Run _ Jog\1113469835.gpx'
_13 = r'C:\DATA\MapMyRun\Barcelona\47114942\16_16_Run _ Jog\1089716703.gpx'
_14 = r'C:\DATA\MapMyRun\Barcelona\54594328\16_16_Run _ Jog\1001316367.gpx'
_15 = r'C:\DATA\MapMyRun\Barcelona\49986800\16_16_Run _ Jog\1096701929.gpx'
_16 = r'C:\DATA\MapMyRun\Barcelona\48076260\16_16_Run _ Jog\1017151539.gpx'
_17 = r'C:\DATA\MapMyRun\Barcelona\28902586\16_16_Run _ Jog\1165040755.gpx'

gpx = gpxpy.parse(open(_3, 'r'))

In [3]:
points_data = gpx.get_points_data()
points = [(pdata.point.latitude, pdata.point.longitude) for pdata in points_data]

In [4]:
tileset = r'http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'
attribution = 'Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> \
                &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'

_map = folium.Map(tiles=tileset, attr=attribution, max_zoom=25)

start_mark = folium.Marker([points[0][0], points[0][1]], icon=folium.Icon(color='green', icon='glyphicon glyphicon-play'))
start_mark.add_to(_map)

end_mark = folium.Marker([points[-1][0], points[-1][1]], icon=folium.Icon(color='red', icon='glyphicon glyphicon-stop'))
end_mark.add_to(_map)

line = folium.PolyLine(points, opacity=0.8)
line.add_to(_map)
_map.fit_bounds(line.get_bounds())

_map

## LinkedGeoData

In [7]:
concepts = "lgdo:Leisure, lgdo:Natural, lgdo:Landuse"
#concepts = "lgdo:Building"
concepts.split(',')

['lgdo:Leisure', ' lgdo:Natural', ' lgdo:Landuse']

In [8]:
# Search for tags in OSN
osn_graph = Graph()
osn_graph.parse('osm_semantic_network.skos.rdf')

lgdo_graph = Graph()
lgdo_graph.parse('lgdo_2014-07-26.n3', format='n3')

<Graph identifier=N9d3a7908766c4fdb8ee09d239d4b5573 (<class 'rdflib.graph.Graph'>)>

In [9]:
lgdo_prefix = 'http://linkedgeodata.org/ontology/'

query = """
PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lgdo:<http://linkedgeodata.org/ontology/>

SELECT *
{
 ?s a skos:Concept .
 ?s skos:exactMatch ?concept .
 FILTER (?concept IN (""" + concepts + """))
}"""


#query.format(osnt_url)
print(query)

response = osn_graph.query(query)
osn_terms = ['<{}>'.format(row[0]) for row in response]
osn_terms


PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lgdo:<http://linkedgeodata.org/ontology/>

SELECT *
{
 ?s a skos:Concept .
 ?s skos:exactMatch ?concept .
 FILTER (?concept IN (lgdo:Leisure, lgdo:Natural, lgdo:Landuse))
}


['<http://spatial.ucd.ie/lod/osn/term/k:landuse>',
 '<http://spatial.ucd.ie/lod/osn/term/k:natural>',
 '<http://spatial.ucd.ie/lod/osn/term/k:leisure>']

In [10]:
#Extend keys with osm_semantic_network.skos.rdf
osn_terms

['<http://spatial.ucd.ie/lod/osn/term/k:landuse>',
 '<http://spatial.ucd.ie/lod/osn/term/k:natural>',
 '<http://spatial.ucd.ie/lod/osn/term/k:leisure>']

In [11]:
#?s skos:related {0} .
query = """
PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>

SELECT ?subject ?relatedTerm ?concept1 ?concept2
{
 ?subject skos:related ?relatedTerm .
 
 ?relatedTerm skos:exactMatch ?concept1 .
 
 ?subject skos:exactMatch ?concept2 .
 
 FILTER (?relatedTerm IN (""" + ', '.join(osn_terms) + """))
 FILTER (STRSTARTS(STR(?concept2), "http://linkedgeodata.org/ontology"^^xsd:string))
}"""

print(query)

response = osn_graph.query(query)
len(response)


PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>

SELECT ?subject ?relatedTerm ?concept1 ?concept2
{
 ?subject skos:related ?relatedTerm .
 
 ?relatedTerm skos:exactMatch ?concept1 .
 
 ?subject skos:exactMatch ?concept2 .
 
 FILTER (?relatedTerm IN (<http://spatial.ucd.ie/lod/osn/term/k:landuse>, <http://spatial.ucd.ie/lod/osn/term/k:natural>, <http://spatial.ucd.ie/lod/osn/term/k:leisure>))
 FILTER (STRSTARTS(STR(?concept2), "http://linkedgeodata.org/ontology"^^xsd:string))
}


94

In [12]:
for row in response:
    #print(row[0], '\t', row[1])
    for element in row:
        print(element)
    print()

http://spatial.ucd.ie/lod/osn/term/k:amenity/v:parking
http://spatial.ucd.ie/lod/osn/term/k:landuse
http://linkedgeodata.org/ontology/Landuse
http://linkedgeodata.org/ontology/Parking

http://spatial.ucd.ie/lod/osn/term/k:sport/v:bowls
http://spatial.ucd.ie/lod/osn/term/k:leisure
http://linkedgeodata.org/ontology/Leisure
http://linkedgeodata.org/ontology/Bowls

http://spatial.ucd.ie/lod/osn/term/k:gambling
http://spatial.ucd.ie/lod/osn/term/k:leisure
http://linkedgeodata.org/ontology/Leisure
http://linkedgeodata.org/ontology/Gambling

http://spatial.ucd.ie/lod/osn/term/k:sport
http://spatial.ucd.ie/lod/osn/term/k:landuse
http://linkedgeodata.org/ontology/Landuse
http://linkedgeodata.org/ontology/Sport

http://spatial.ucd.ie/lod/osn/term/k:sport/v:swimming
http://spatial.ucd.ie/lod/osn/term/k:leisure
http://linkedgeodata.org/ontology/Leisure
http://linkedgeodata.org/ontology/Swimming

http://spatial.ucd.ie/lod/osn/term/k:landuse/v:meadow
http://spatial.ucd.ie/lod/osn/term/k:natural
http

In [13]:
gpx_simple = gpx.clone()
gpx_simple.simplify()

linestring_simple = "LINESTRING("

simple_points = gpx_simple.get_points_data()

for i, point_data in enumerate(simple_points):
    point = point_data.point
    linestring_simple += str(point.longitude) + " " + str(point.latitude)
    if i < len(simple_points)-1:
        linestring_simple += ", "

linestring_simple += ")"
linestring_simple

'LINESTRING(5.73425791 45.18210998, 5.7353382904 45.1824220976, 5.7408826096 45.1847253391, 5.7429230865 45.186140418, 5.7424419076 45.1868992116, 5.7424473461 45.1880505119, 5.7428454475 45.1889469214, 5.7430898764 45.1890251298, 5.7437873918 45.1889355192, 5.7451961052 45.1884058183, 5.7454810635 45.1883963889, 5.7481449371 45.188360427, 5.7494157359 45.1884815264, 5.7504363 45.1888904565, 5.7492650439 45.1885329393, 5.7468814271 45.1884723829, 5.7453555906 45.1887051998, 5.743829997 45.1891110944, 5.7429074605 45.1889185302, 5.7425774415 45.1883845097, 5.742458776 45.1875742611, 5.7422837729 45.1872283783, 5.7425883036 45.1864469882, 5.7426182096 45.185734623, 5.7422678227 45.1855394129)'

In [14]:
endpoint = 'http://linkedgeodata.org/sparql'
sparql = SPARQLWrapper2(endpoint) 
sparql.setReturnFormat('json')

distance = 0.001 # 1 meter

query = """
PREFIX ogc: <http://www.opengis.net/ont/geosparql#>
PREFIX geom: <http://geovocab.org/geometry#>
PREFIX lgdo: <http://linkedgeodata.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
FROM <http://linkedgeodata.org> {
  ?subject
    a ?type ;
    geom:geometry [
      ogc:asWKT ?geometry
    ] .
    OPTIONAL { 
        ?subject rdfs:label ?label .
    }
    OPTIONAL { 
        ?subclass rdfs:subClassOf ?type .
        ?subject a ?subclass . 
    }
    OPTIONAL { 
        ?type rdfs:subClassOf ?superclass .
    }
    FILTER (?type IN (""" + concepts + """))
    FILTER (bif:st_intersects (?geometry, 
    bif:st_geomFromText(" """ + linestring_simple + """ "), 
   """ + str(distance) + """))
}"""

print(query)

sparql.setQuery(query)
results = sparql.query().convert()
dict_array = []
for dic in results.bindings:
    new_dict = {k: v.value for k, v in dic.items()}
    dict_array.append(new_dict)
    
df = pnd.DataFrame(dict_array)
df


PREFIX ogc: <http://www.opengis.net/ont/geosparql#>
PREFIX geom: <http://geovocab.org/geometry#>
PREFIX lgdo: <http://linkedgeodata.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
FROM <http://linkedgeodata.org> {
  ?subject
    a ?type ;
    geom:geometry [
      ogc:asWKT ?geometry
    ] .
    OPTIONAL { 
        ?subject rdfs:label ?label .
    }
    OPTIONAL { 
        ?subclass rdfs:subClassOf ?type .
        ?subject a ?subclass . 
    }
    OPTIONAL { 
        ?type rdfs:subClassOf ?superclass .
    }
    FILTER (?type IN (lgdo:Leisure, lgdo:Natural, lgdo:Landuse))
    FILTER (bif:st_intersects (?geometry, 
    bif:st_geomFromText(" LINESTRING(5.73425791 45.18210998, 5.7353382904 45.1824220976, 5.7408826096 45.1847253391, 5.7429230865 45.186140418, 5.7424419076 45.1868992116, 5.7424473461 45.1880505119, 5.7428454475 45.1889469214, 5.7430898764 45.1890251298, 5.7437873918 45.1889355192, 5.7451961052 45.1884058183, 5.7454810635 45.1883963889, 5.74814

,geometry,label,subclass,subject,type
0,"LINESTRING(5.7508829 45.1834449,5.7513476 45.1...",Gymnase Auguste Delaune,http://linkedgeodata.org/ontology/SportsCentre,http://linkedgeodata.org/triplify/way187151639,http://linkedgeodata.org/ontology/Leisure
1,"LINESTRING(5.7522829 45.182372,5.7509442 45.18...",Stade Auguste Delaune,http://linkedgeodata.org/ontology/Pitch,http://linkedgeodata.org/triplify/way25617464,http://linkedgeodata.org/ontology/Leisure
2,"LINESTRING(5.7514139 45.1854591,5.751773 45.18...",NaN,http://linkedgeodata.org/ontology/Pitch,http://linkedgeodata.org/triplify/way43950848,http://linkedgeodata.org/ontology/Leisure
3,"LINESTRING(5.739969 45.1893628,5.7396816 45.18...",Bike Park du Clochard,http://linkedgeodata.org/ontology/Pitch,http://linkedgeodata.org/triplify/way237749396,http://linkedgeodata.org/ontology/Leisure
4,"LINESTRING(5.7391935 45.1875056,5.74053 45.188...",NaN,http://linkedgeodata.org/ontology/Pitch,http://linkedgeodata.org/triplify/way25532414,http://linkedgeodata.org/ontology/Leisure
5,"LINESTRING(5.7507687 45.1838562,5.7507881 45.1...",Piscine municipale,http://linkedgeodata.org/ontology/SwimmingPool,http://linkedgeodata.org/triplify/way4358857,http://linkedgeodata.org/ontology/Leisure
6,"LINESTRING(5.7427155 45.181794,5.7426464 45.18...",Moana Club,http://linkedgeodata.org/ontology/FitnessCentre,http://linkedgeodata.org/triplify/way77363386,http://linkedgeodata.org/ontology/Leisure


## Overpass API

In [15]:
keys = ["leisure", "natural", "landuse"]
#keys = ["."]

In [16]:
osm_features = osm.get_spatial_features(gpx, keys)

In [17]:
print(len(osm_features))
osm_features

9


,geometry,label,relation,tags
id,,,,
278498,"MULTILINESTRING((5.6147474 45.2952120,5.614575...",L'Isère,None,"{'natural': 'water', 'note': 'taggé en mode « ..."
4014810,"LINESTRING(5.7357462 45.1826746, 5.7391442 45....",Parc Paul Mistral,[],"{'name': 'Parc Paul Mistral', 'leisure': 'park'}"
22981584,"LINESTRING(5.7398870 45.1852845, 5.7402628 45....",Palais des Sports,[],"{'name': 'Palais des Sports', 'leisure': 'spor..."
28340980,"LINESTRING(5.7381446 45.1908332, 5.7385147 45....",Parc des Berges de l'Isère,[],"{'name': 'Parc des Berges de l'Isère', 'alt_na..."
28699752,"LINESTRING(5.7360523 45.1833667, 5.7369516 45....",Halle Clemenceau,[],"{'name': 'Halle Clemenceau', 'building': 'yes'..."
183471219,"LINESTRING(5.7386597 45.1846293, 5.7390029 45....",None,[],{'landuse': 'forest'}
183471265,"LINESTRING(5.7383389 45.1852124, 5.7393029 45....",None,[],{'landuse': 'grass'}
382417716,"LINESTRING(5.7426615 45.1879540, 5.7435489 45....",Châtelet Abbaye Jouhaux,[],"{'name': 'Châtelet Abbaye Jouhaux', 'quarter':..."
440478753,"LINESTRING(5.7425873 45.1896088, 5.7425129 45....",None,[],{'natural': 'wood'}


In [18]:
#for feature in osm_features.itertuples():
#    print(type(feature.tags))
#    print(feature.tags)
#    for tag in feature.tags.items():
#        print(tag)
osn_terms = []
exclude = ['name', 'source', 'comment', 'boundary']
osn_url_model = "<http://spatial.ucd.ie/lod/osn/term/k:{0}/v:{1}>"
for tag in osm_features['tags']:
    for item in tag.items():
        if not any([term in item[0] for term in exclude]):
            osn_term = osn_url_model.format(item[0], item[1].replace(' ', '_'))
            print(osn_term)
            osn_terms.append(osn_term)

len(set(osn_terms))

<http://spatial.ucd.ie/lod/osn/term/k:natural/v:water>
<http://spatial.ucd.ie/lod/osn/term/k:note/v:taggé_en_mode_«_new_tagging_»_cf_wiki>
<http://spatial.ucd.ie/lod/osn/term/k:type/v:multipolygon>
<http://spatial.ucd.ie/lod/osn/term/k:water/v:river>
<http://spatial.ucd.ie/lod/osn/term/k:boat/v:no>
<http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park>
<http://spatial.ucd.ie/lod/osn/term/k:leisure/v:sports_centre>
<http://spatial.ucd.ie/lod/osn/term/k:website/v:http://www.palais-des-sports.com/>
<http://spatial.ucd.ie/lod/osn/term/k:building/v:yes>
<http://spatial.ucd.ie/lod/osn/term/k:phone/v:+33_4_76_54_27_27>
<http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park>
<http://spatial.ucd.ie/lod/osn/term/k:building/v:yes>
<http://spatial.ucd.ie/lod/osn/term/k:leisure/v:sports_centre>
<http://spatial.ucd.ie/lod/osn/term/k:landuse/v:forest>
<http://spatial.ucd.ie/lod/osn/term/k:landuse/v:grass>
<http://spatial.ucd.ie/lod/osn/term/k:quarter/v:suburb>
<http://spatial.ucd.ie/lod/osn/term/k:landu

15

In [19]:
#?s skos:related {0} .
query = """
PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>

SELECT ?subject ?concept
{
 ?subject skos:exactMatch ?concept .
 
 FILTER (?subject IN (""" + ', '.join(set(osn_terms)) + """))
 FILTER (STRSTARTS(STR(?concept), "http://linkedgeodata.org/ontology"^^xsd:string))
}"""

print(query)

response = osn_graph.query(query)
len(response)


PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>

SELECT ?subject ?concept
{
 ?subject skos:exactMatch ?concept .
 
 FILTER (?subject IN (<http://spatial.ucd.ie/lod/osn/term/k:water/v:river>, <http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park>, <http://spatial.ucd.ie/lod/osn/term/k:leisure/v:sports_centre>, <http://spatial.ucd.ie/lod/osn/term/k:note/v:taggé_en_mode_«_new_tagging_»_cf_wiki>, <http://spatial.ucd.ie/lod/osn/term/k:boat/v:no>, <http://spatial.ucd.ie/lod/osn/term/k:building/v:yes>, <http://spatial.ucd.ie/lod/osn/term/k:website/v:http://www.palais-des-sports.com/>, <http://spatial.ucd.ie/lod/osn/term/k:landuse/v:grass>, <http://spatial.ucd.ie/lod/osn/term/k:natural/v:wood>, <http://spatial.ucd.ie/lod/osn/term/k:type/v:multipolygon>, <http://spatial.ucd.ie/lod/osn/term/k:quarter/v:suburb>, <http://spatial.ucd.ie/lod/o

7

In [20]:
selected_terms = []
for row in response:
    selected_terms.append(row[0])
    for element in row:
        print(element)
        
    print()
    
set(selected_terms)

http://spatial.ucd.ie/lod/osn/term/k:landuse/v:forest
http://linkedgeodata.org/ontology/LanduseForest

http://spatial.ucd.ie/lod/osn/term/k:natural/v:wood
http://linkedgeodata.org/ontology/NaturalWood

http://spatial.ucd.ie/lod/osn/term/k:natural/v:water
http://linkedgeodata.org/ontology/NaturalWater

http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park
http://linkedgeodata.org/ontology/Park

http://spatial.ucd.ie/lod/osn/term/k:landuse/v:grass
http://linkedgeodata.org/ontology/Grass

http://spatial.ucd.ie/lod/osn/term/k:leisure/v:sports_centre
http://linkedgeodata.org/ontology/SportsCentre

http://spatial.ucd.ie/lod/osn/term/k:landuse/v:residential
http://linkedgeodata.org/ontology/LanduseResidential



{rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:landuse/v:forest'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:landuse/v:grass'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:landuse/v:residential'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:leisure/v:sports_centre'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:natural/v:water'),
 rdflib.term.URIRef('http://spatial.ucd.ie/lod/osn/term/k:natural/v:wood')}

In [21]:
selected_tags = []
for term in selected_terms:
    elements = term.split('/')
    key = elements[-2].split(':')[-1]
    value = elements[-1].split(':')[-1]
    
    selected_tags.append((key, value))
    
#selected_tags = np.array(selected_tags)
selected_tags

[('landuse', 'forest'),
 ('natural', 'wood'),
 ('natural', 'water'),
 ('leisure', 'park'),
 ('landuse', 'grass'),
 ('leisure', 'sports_centre'),
 ('landuse', 'residential')]

In [22]:
selected_features_i = []
for tag, idx in zip(osm_features['tags'], osm_features['tags'].index):
    for item in tag.items():
        if item in selected_tags:
            print(idx, 'match', item)
            selected_features_i.append(idx)
            
selected_features_i = set(selected_features_i)
len(selected_features_i)

278498 match ('natural', 'water')
4014810 match ('leisure', 'park')
22981584 match ('leisure', 'sports_centre')
28340980 match ('leisure', 'park')
28699752 match ('leisure', 'sports_centre')
183471219 match ('landuse', 'forest')
183471265 match ('landuse', 'grass')
382417716 match ('landuse', 'residential')
440478753 match ('natural', 'wood')


9

In [23]:
osm_features.loc[selected_features_i]

,geometry,label,relation,tags
id,,,,
183471265,"LINESTRING(5.7383389 45.1852124, 5.7393029 45....",None,[],{'landuse': 'grass'}
278498,"MULTILINESTRING((5.6147474 45.2952120,5.614575...",L'Isère,None,"{'natural': 'water', 'note': 'taggé en mode « ..."
440478753,"LINESTRING(5.7425873 45.1896088, 5.7425129 45....",None,[],{'natural': 'wood'}
28699752,"LINESTRING(5.7360523 45.1833667, 5.7369516 45....",Halle Clemenceau,[],"{'name': 'Halle Clemenceau', 'building': 'yes'..."
22981584,"LINESTRING(5.7398870 45.1852845, 5.7402628 45....",Palais des Sports,[],"{'name': 'Palais des Sports', 'leisure': 'spor..."
183471219,"LINESTRING(5.7386597 45.1846293, 5.7390029 45....",None,[],{'landuse': 'forest'}
28340980,"LINESTRING(5.7381446 45.1908332, 5.7385147 45....",Parc des Berges de l'Isère,[],"{'name': 'Parc des Berges de l'Isère', 'alt_na..."
382417716,"LINESTRING(5.7426615 45.1879540, 5.7435489 45....",Châtelet Abbaye Jouhaux,[],"{'name': 'Châtelet Abbaye Jouhaux', 'quarter':..."
4014810,"LINESTRING(5.7357462 45.1826746, 5.7391442 45....",Parc Paul Mistral,[],"{'name': 'Parc Paul Mistral', 'leisure': 'park'}"


## Map

In [24]:
linestring = "LINESTRING("

for i, point_data in enumerate(points_data):
    point = point_data.point
    linestring += str(point.longitude) + " " + str(point.latitude)
    if i < len(points_data)-1:
        linestring += ", "

linestring += ")"

trajectory = wkt.loads(linestring)

buffer_size = 0.0002 # ~20 meters

features = []
features_json = []
marker_data = []

for item in osm_features.itertuples(): #########
    feature0 = wkt.loads(item.geometry)
    json_feature0 = geojson.Feature(geometry=feature0, properties={"id": str(item.Index)})

    feature1 = feature0.buffer(buffer_size)
    json_feature1 = geojson.Feature(geometry=feature1)

    feature2 = polygonize_full(feature0)[0]
    json_feature2 = geojson.Feature(geometry=feature2)

    feature3 = cascaded_union([feature1, feature2])
    json_feature3 = geojson.Feature(geometry=feature3)

    #0: original geometry (linestring)
    #1: buffered 0 (polygon)
    #2: polygonized
    #3: union of 1 and 2
    # This is for visualization purposes.
    # A more efficient solution would be to buffer the polygonized line (feature2)
    features_json.append([json_feature0, json_feature1, json_feature2, json_feature3])
    features.append(feature3)

    rep_point = feature3.representative_point()
    popup_text = repr(item.tags)
    if item.relation and len(item.relation) > 0:
        popup_text += " | part of: " + str(item.relation)
    marker_data.append(([rep_point.y, rep_point.x], popup_text))

In [25]:
tileset = r'http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'
attribution = 'Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> \
                &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'

traj_map = folium.Map(tiles=tileset, attr=attribution, max_zoom=25)

for i, feat in enumerate(features_json):
    traj_map.choropleth(geo_str=feat[0], line_color='black', line_weight=3)
    #traj_map.choropleth(geo_str=feat[1], fill_color='gray', line_weight=1, fill_opacity=0.2)
    #traj_map.choropleth(geo_str=feat[2], fill_color='green', line_weight=1, fill_opacity=0.2)
    traj_map.choropleth(geo_str=feat[3], fill_color='red', line_weight=1, fill_opacity=.1)
    folium.Marker(location=marker_data[i][0], popup=marker_data[i][1]).add_to(traj_map)

start_mark.add_to(traj_map)
end_mark.add_to(traj_map)

line.add_to(traj_map)

traj_map.fit_bounds(line.get_bounds())

traj_map

In [26]:
feature_intervals = osm.compute_intersections(gpx, osm_features) ############
print("{0} features retrieved.".format(len(feature_intervals)))
feature_intervals

19 features retrieved.


,duration_ratio,end_index,interval_duration,interval_length,length_ratio,start_index
id,,,,,,
278498,0.033708,114,33.0,89.143794,0.031354,106
278498,0.109295,151,107.0,310.037116,0.109048,124
278498,0.205312,208,201.0,575.861797,0.202546,158
4014810,0.233912,71,229.0,694.994346,0.244448,10
4014810,0.015322,81,15.0,49.119644,0.017277,77
4014810,0.000000,230,0.0,0.000000,0.000000,230
4014810,0.041879,248,41.0,116.845211,0.041097,238
22981584,0.008172,57,8.0,21.842304,0.007683,55
28340980,0.167518,126,164.0,495.021849,0.174112,83


## Group by id

In [27]:
groups = feature_intervals.groupby(level=0)
print(len(groups), 'unique features.')

9 unique features.


## Select features

In [28]:
minimum_length_ratio = 0.2
minimum_duration_ratio = 0.2

In [29]:
selected = []
for group in groups:
    if group[1]['length_ratio'].sum() > minimum_length_ratio or group[1]['duration_ratio'].sum() > minimum_duration_ratio:
        #tag_present = False
        #for key in osm_features.loc[group[0]]['tags'].keys():
        #    if key in keys:
        #        tag_present = True
        #        continue

        #if tag_present:
            print('')
            print(group[0])
            print('relations:', osm_features.loc[group[0]]['relation'])
            print(osm_features.loc[group[0]]['label'], osm_features.loc[group[0]]['tags'])
            print("length ratio: {:.2f}%".format(group[1]['length_ratio'].sum()*100))
            print("duration ratio: {:.2f}%".format(group[1]['duration_ratio'].sum()*100))
            selected.append(group[0])


278498
relations: None
L'Isère {'natural': 'water', 'note': 'taggé en mode « new tagging » cf wiki', 'type': 'multipolygon', 'name': "L'Isère", 'source': 'cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre;mise à jour : 2009', 'water': 'river', 'comment': 'Provenance du cadastre, les bords semblent correspondre au seuil maximal innondable', 'boat': 'no'}
length ratio: 34.29%
duration ratio: 34.83%

4014810
relations: []
Parc Paul Mistral {'name': 'Parc Paul Mistral', 'leisure': 'park'}
length ratio: 30.28%
duration ratio: 29.11%

28340980
relations: []
Parc des Berges de l'Isère {'name': "Parc des Berges de l'Isère", 'alt_name': 'Parc Paul Mistral', 'leisure': 'park'}
length ratio: 35.30%
duration ratio: 33.40%

382417716
relations: []
Châtelet Abbaye Jouhaux {'name': 'Châtelet Abbaye Jouhaux', 'quarter': 'suburb', 'landuse': 'residential'}
length ratio: 32.66%
duration ratio: 32.48%

440478753
relations: []
None {'natural': 'wood'}
length ratio: 42.48%
duration ratio: 

## Map of selected features

In [30]:
selected_map = folium.Map(tiles=tileset, attr=attribution, max_zoom=25)

for osm_index in selected:
    index = osm_features.index.get_loc(osm_index)
    selected_map.choropleth(geo_str=features_json[index][3], line_weight=4, 
                            line_opacity=.6, line_color='black', fill_color='red', fill_opacity=.1)
    folium.Marker(location=marker_data[index][0], popup=marker_data[index][1]).add_to(selected_map)

start_mark.add_to(selected_map)
end_mark.add_to(selected_map)
line.add_to(selected_map)

selected_map.fit_bounds(line._get_self_bounds())

selected_map

## Create STEP instances

In [31]:
#for key, value in osm_features.loc[group[0]]['tags'].iteritems():
#    if key in keys:
#        print '{0}k:{1}/v:{2}'.format(osn_namespace, key, value)
# building=yes
#  (leisure|natural|landuse)*=(water|riverbank|residential|park)*
# merge
# http://spatial.ucd.ie/lod/osn/term/
#'''
#PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

#SELECT DISTINCT * 
#{ 
# GRAPH <http://OSN> {
#  <http://spatial.ucd.ie/lod/osn/term/k:leisure/v:park> skos:related ?related .
# }
#}

In [32]:
OSNT = Namespace('http://spatial.ucd.ie/lod/osn/term/')
LGDR = Namespace('http://linkedgeodata.org/triplify/')
EXAMPLE = Namespace('http://example.com/resource/')

raw = utility.create_raw_trajectory(gpx)

In [33]:
foi_places = step.FeatureOfInterest("Places")

for group in groups:
    if group[0] in selected:
        intervals = group[1]
        feature_id = group[0]
        
        semantics_str = 'near'
        if osm_features.loc[feature_id].label:
            semantics_str += ' ' +osm_features.loc[feature_id].label
        place_semantics = step.QualitativeDescription(semantics_str)
        
        if osm_features.loc[feature_id].relation is None:
            element_type = 'relation'
        else:
            element_type = 'way'

        osm_label = str(osm_features.loc[feature_id]['label'])
        
        related_tags = []
        for key, value in osm_features.loc[feature_id]['tags'].items():
            if key in keys:
                related_tags.append(URIRef("{0}k:{1}/v:{2}".format(OSNT, key, value)))
                
        lgdo_uri = URIRef("{0}{1}{2}".format(LGDR, element_type, feature_id))
        
        contextual_element = step.ContextualElement(osm_label, lgdo_uri, related_tags)
        
        for item in intervals.itertuples():
            start_fix = raw.get_fix(points_data[item.start_index].point.time)
            end_fix = raw.get_fix(points_data[item.end_index].point.time)
            st_extent = step.SpatiotemporalExtent(start_fix, end_fix)

            episode = step.Episode(st_extent, place_semantics)
            episode.relate(contextual_element)

            foi_places.add_episode(episode)

#print(foi_places)

In [34]:
triples = foi_places.triplify()
#print(triples)

In [35]:
g = rdflib.Graph()
g.bind('step', namespaces.STEP)
g.bind('skos', namespaces.SKOS)
g.bind('geo', namespaces.GEO)
g.bind('time', namespaces.TIME)
g.bind('owl', namespaces.OWL)

for triple in triples:
    g.add(triple)
    
g.serialize("geographic_features_of_interest.rdf")